## Set-Up

In [1]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, inspect

In [2]:
# Create an engine for the `wine_db.sqlite` database
engine = create_engine("sqlite:///Resources/wine_data.db")

In [3]:
inspector = inspect(engine)
inspector.get_table_names()

['winequalityRed', 'winequalityWhite']

In [4]:
white_wine = engine.execute('SELECT * FROM winequalityWhite').fetchall()

In [5]:
df = pd.DataFrame(white_wine, columns=["wine_id","fixed acidity","volatile acidity", "citric acid",
                                "residual sugar","chlorides","free sulfur dioxide","total sulfur dioxide",
                                "density","pH","sulphates","alcohol","quality"])

In [6]:
# Separate feature variables and target variable
X = df.drop(['quality','wine_id'], axis = 1)
y = df['quality']

In [7]:
# See proportion of good vs bad wines
df['quality'].value_counts()

1    3258
0    1640
Name: quality, dtype: int64

In [8]:
# Normalize feature variables
from sklearn.preprocessing import StandardScaler
X_features = X
scalar = StandardScaler().fit(X)
X = StandardScaler().fit_transform(X)

In [9]:
# Splitting the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=0)

In [10]:
from sklearn.metrics import classification_report
from sklearn.ensemble import ExtraTreesClassifier
model = ExtraTreesClassifier(random_state=1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.67      0.75       441
           1       0.83      0.93      0.88       784

    accuracy                           0.84      1225
   macro avg       0.84      0.80      0.81      1225
weighted avg       0.84      0.84      0.83      1225



In [11]:
import pickle
pkl_filename = "app/white_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model,file)

scaler = StandardScaler().fit(X)
pkl_scaled_filename = 'app/scaler_white.pkl'
with open(pkl_scaled_filename, 'wb') as file:
    pickle.dump(scaler,file)


In [12]:
# Function to Load and run model with input scores

def white_model(user_input_array):
    import pickle
    #Load Model
    with open('app/white_model.pkl', 'rb') as file:
        white_model = pickle.load(file)
    with open('app/scaler_white.pkl', 'rb') as file:
        scaler = pickle.load(file)

    X = user_input_array.reshape(1,-1)
    X_scaled = scaler.transform(X)

    # Calculate Wine quality
    y_pred_white = white_model.predict(X_scaled)

    return y_pred_white

In [13]:
test = X_test[1]
answer = y[1]
print(test, answer)

[ 0.17209696  0.31511748 -0.6131152   0.19887157  1.06325748 -0.42975081
  0.50923618  0.45897907 -0.38590965 -0.26155273 -0.58047134] 1


In [14]:
white_model(test)

array([1])